# The Monte-Carlo method
<hr>

## Summary
The Monte-Carlo method is an important numerical technique that involves exploiting the uniform nature of random processes. By comparison to many other numerical techniques it is juvenile in age, naive by default, but of incredible general ability. Here the Monte-Carlo method is introduced with historical and mathematical context in the introduction section, a pseudorandom object generator is written out with Python 3 in the implementation section, which is utilized in the application section to evaluate a simple definite integral, and approximate the eigenspace of a small matrix.

## Introduction
Before diving into the technical details of the Monte-Carlo method (MC), it is worth noting a few historical details. The MC's systematic development dates back to about 1944, but interesting expositions such as the Buffon's needle approximation of pi predate this year. The MC has helped humans: design fission-based explosive devices for exploding each other, optimize parcel delivery routes and other complex business strategies, and, most-crucially, implement the digital ["Magic Eight Ball"](https://u-ts.net/eightball). Whereas it's conception and usage appears to be centered on the perpetuation of human misery, the folks who were comissioned or compelled to do the actual development were otherwise very bright, interesting, and nice (see, e.g., [Stan Ulam](https://en.wikipedia.org/wiki/Stanislaw_Ulam), [Enrico Fermi](https://en.wikipedia.org/wiki/Enrico_Fermi), [John von Neumann](https://en.wikipedia.org/wiki/John_von_Neumann)). For more information regarding the MC's history, refer to JM Hammersley and DC Handscomb's 1964 title _Monte Carlo Methods,_ which this introduction follows.

Hammersley and Handscomb define the MC as _that branch of experimental mathematics which is concerned with experiments on random numbers._ There are many forms of this experimentation, but for simplicity, suppose the experiment is to simply evalute

$$\frac{1}{\sqrt{2\pi}}\int_{0}^{100}e^{-\frac{x^{2}}{2}}dx.$$

Most "deterministic" numerical algorithms (e.g., Simpson's method, trapezoid method) usually begin at the lower limit ($0$ in this case), and iterate toward the upper limit, serially adding up slice areas. In contrast to this paradigm, the MC works by choosing at random slices between (and of course including) the integration limits to perform it's serial addition. That is how the MC works-- it is the _process_ of picking at random things and doing stuff with those things until some break condition is met. The random process in this experiment is picking values in the integral's domain to use in it's approximation.

The MC however is in certain settings very "dumb". If it were implemented to evaluate the above integral, it would be terribly non-performant because most of the integral's value is between $x=0$ and $x=2$. Numerical algorithms that start at the lower limit and traverse serially toward the upper limit will have quickly produced an accurate estimate, but the MC in contrast will just be randomly "walking around", adding up slices picked uniformly throughout the domain. To circumvent this lack of convergence in practical applications, MC implementations are typically _biased_ with additional logic to drive up performance. Note as well that there are situations where the only feasible "method of experimentation" is exposed by the MC, such as statistical sampling, optimization, and simulation.

With this basic exposition of the MC, what exactly is a random process? Sadly, there is no single "authoritative" definition for random processes, but two important characteristics are that the objects "picked" or "visited" by the process _distribute uniformly_, and the mechanics of "picking/visiting the next object" do not involve the objects already picked.

Interestingly, humans have not yet formulated or implemented an algorithm for random processes that can pick objects indefinitely, such that the picked objects distribute uniformly and are uncorrelated (exercise: guess why this is the case). At the moment, existing algorithms designed to mimic a random processes are _periodic_, meaning they eventually cycle back to prior behavior. Thus these algorithms are said to be _pseudorandom_.



## Implementation

In [1]:
from warnings import warn


class Mersenne:
    """Pseudorandom object generater"""

    def __init__(self, seed=1234):
        """Construct pseudorandom computation attributes"""
        self.seed = seed
        self.j = 2**31 - 1
        self.k = 16807
        self.period = 2**30

    def floating(self, interval=None, count=1):
        """
        Return a pseudorandom float. Default is one floating-
        point number between zero and one. Pass in a tuple or
        list, (a,b), to return a floating-point number on
        [a,b]. If count is 1, a single number is returned,
        otherwise a list of numbers is returned.
        """
        results = []
        if interval is not None:
            start, interval = interval[0], interval[1] - interval[0]
        else:
            start, interval = 0.0, 1.0
        for i in range(count):
            self.seed = (self.k * self.seed) % self.j
            results.append((interval * (self.seed / self.j)) + start)
            self.period -= 1
            if self.period == 0:
                warn("Pseudorandom period warning!!")
        return results
    
    def integer(self, interval=None, count=1):
        """
        Return a pseudorandom integer. Default is one integer
        number in {0,1}. Pass in a tuple or list, (a,b), to
        return an integer number on [a,b]. If count is 1, a
        single number is returned, otherwise a list of numbers
        is returned.
        """
        if count == 1:
            return [round(self.floating(interval=interval).pop())]
        elif count > 1:
            return [
                round(f) for f in self.floating(
                    interval=interval, count=count
                )
            ]
            
    def alpha(self, count=1):
        """
        Return a pseudorandom alphabetical letter (English).
        By default, one letter in a-z is returned. If count is
        greater than 1, a string a length count is returned.
        """
        chars = "abcdefghijklmnopqrstuvwxyz"
        if count == 1:
            return chars[self.integer(interval=(0,25))]
        elif count > 1:
            return "".join(
                chars[f] for f in self.integer(
                    interval=(0,25), count=count
                )
            )
            
            
random = Mersenne()

## Applications

### Integration
Consider the integral

$$I = \int_{0}^{\pi} \sin(x)dx,$$

whose analytical value is $2$. It can be shown that,

$$I = \pi\lim_{N\rightarrow\infty}\frac{1}{N}\sum_{n=0}^{N-1}\sin(x_{n}),$$

whereby the lack of order and possbile redundancies in the set of $x_{n}$ values provides the key hook for implmenting the MC. Notice that the RHS expression is multiplied by $\pi$. In fact, the general form is scaled by the difference between the limits of integration, like so:

$$I = \int_{a}^{b} f(x)dx = (b - a)\lim_{N\rightarrow\infty}\frac{1}{N}\sum_{n=0}^{N-1}f(x_{n})$$

So then, to estimate a definite inegral of a single variable with the MC, the following method accepts an iterable of integration limits (of the form `[lower, upper]`) and a function, and uses `Mersenne.floating` to evaluate the function with a random variate in the domain specified by the limits. Each result is then stored in a list, and every time the list's length is a clean multiple of $100$, an `if` block can break the loop if the average is stationary (exercise: describe how).

In [2]:
def mc_integrate(limits, function):
    """Return definite integral approximation"""
    chain = [function(random.floating(limits).pop())]
    prior_avg = sum(chain) / len(chain)
    while True:
        chain.append(function(random.floating(limits).pop()))
        if len(chain) % 100 == 0:
            avg = sum(chain) / len(chain)
            if abs(prior_avg - avg) < 10**(-8):
                return (limits[-1] - limits[0]) * avg
            prior_avg = avg

As can be seen, the `if` block that breaks the loop can be changed to alter the approximation accuracy. With the existing setting, the function is quite inefficient, and as the need accuracy heightens, it will only grow more inefficient. The approximation of $\int_{0}^{\pi} \sin(x)dx$ below is wrapped with calls to `time.perf_counter` to compute the number of seconds required, and this duration is printed along with the approximation itself.

In [3]:
from math import pi, sin
from time import perf_counter


# perform integration
start_time = perf_counter()
value = mc_integrate((0.0, pi), sin)
end_time = perf_counter()


# show the results
print(end_time - start_time, value)

3.3663530989999995 2.0000081165643873


This is unacceptibly bad performance. In fact, the error between `mc_integrate`'s approximation and the integral's true value goes as $\frac{1}{\sqrt{N}}$ (exercise: derive this asymptotic expression).

### Approximating eigenspaces


In [ ]:
from math import cos


A = [[], []]


#  theta = arccos(<xA,x^T> / ||x|| ||Ax||)


## References and further reading
JM Hammersley, DC Handscomb (1964). _Monte Carlo Methods_. Methuen and Company LTD.

